# AIVLE스쿨 4기 DX트랙 5차 미니프로젝트 
## [미션#1] API를 이용하여 공공 데이터 가져오기

 
 - <font size =+1> 공공데이터포털(https://data.go.kr)에서 제공되는 [국립중앙의료원 - 전국응급의료기관 조회 서비스]에서 필요한 자료를 API를 통해 가져옵니다.
 - <font size =+1> 회원 가입 후 해당 자료에 [활용 신청]을 한 뒤, 마이페이지에서 일반 인증키(encoding)을 확인합니다. (PDF 교재 참고)
 - <font size =+1> 대구광역시 중심으로 데이터 추출 
 

 * 사전 확인 사항 
   * haversine 설치
   * python : 3.11.3 (아나콘다 2023.07 버전 설치 기준)

In [180]:
# 필요한 라이브러리 불러오기

import pandas as pd
import numpy as np
from urllib.parse import quote

In [181]:
# 코드 수정 없이 실행

import ssl
from urllib.request import urlopen
context=ssl.create_default_context()
context.set_ciphers("DEFAULT")


* 공공데이터 수집을 위한 기본값 설정

In [182]:
# 개인 인증키 저장
# 공공데이터포털 - 활용신청 : 일반인증키
key = "dmxRDNK9srMd2YgqgrNr94Qwm66GQ2qOeaR52ewsjheCoPNcAzknauVpluJX/EIgKY0Vo85YDDvCD23VN6MdvQ=="

# 조회할 도시 지정
# Request Parameter : 한글 -> quote()인코딩
city = quote('대구광역시')

### 1) 대구광역시 응급의료기관 목록정보 조회하기

 * 주소 (대구광역시)를 기준으로 응급 의료 기관 목록 정보 조회
 * 기관ID, 주소, 기관명, 응급실전화, 병원 위치(경도/위도) 정보를 확인

In [183]:
# 대구광역시내이 응급의료기관 목록정보 조회하기
# url : https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire
# 인증키 지정 : serviceKey = key  ## 갱
# 도시 지정 : Q0 = city
# 출력 행수 : numOfRows = 100 
# page 번호 : pageNo = 1

url = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire' + '?serviceKey=' + key + '&Q0=' + city + '&pageNo=1&numOfRows=100'

print(url) # 만든 url 출력

result = urlopen(url, context = context)
emrList = pd.read_xml(result, xpath='.//item')
emrList.head()



https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire?serviceKey=dmxRDNK9srMd2YgqgrNr94Qwm66GQ2qOeaR52ewsjheCoPNcAzknauVpluJX/EIgKY0Vo85YDDvCD23VN6MdvQ==&Q0=%EB%8C%80%EA%B5%AC%EA%B4%91%EC%97%AD%EC%8B%9C&pageNo=1&numOfRows=100


,dutyAddr,dutyEmcls,dutyEmclsName,dutyName,dutyTel1,dutyTel3,hpid,phpid,rnum,wgs84Lat,wgs84Lon
0,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",G007,지역응급의료기관,(재)미리내천주성삼성직수도회천주성삼병원,053-790-1000,053-790-0555,A1300081,A1300081,1,35.840225,128.705733
1,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",G007,지역응급의료기관,강남병원,053-980-9000,053-980-9019,A1300018,A1300018,2,35.883009,128.663456
2,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",G001,권역응급의료센터,경북대학교병원,053-200-5114,053-200-5100,A1300002,A1300002,3,35.866235,128.604315
3,"대구광역시 중구 달성로 56, 계명대학교대구동산병원 (동산동)",G007,지역응급의료기관,계명대학교대구동산병원,053-250-8114,053-250-8481,A1303386,A1303386,4,35.869460,128.582931
4,대구광역시 달서구 달구벌대로 1035 (신당동),G006,지역응급의료센터,계명대학교동산병원,1577-6622,053-258-6301,A1300003,A1300003,5,35.853887,128.480130


In [184]:
# 응급의료기관 수 확인하기 : shape 
emrList.shape


(21, 11)

In [ ]:
## 응급의료기관 목록정보 중 기관코드(hpid), 응급기관명(dutyName), 주소(dutyAddr), 응급실연락처(dutyTel3), 위도(wgs84Lat), 경도(wgs84Lon) 정보만 추출
## emrList 중 [hpid, dutyName, dutyAddr, dutyTel3, wgs84Lat, wgs84Lon] 컬럼 활용
## solution_df 에 저장하기
solution_df = emrList[['hpid', 'dutyName', 'dutyAddr', 'dutyTel3', 'wgs84Lat', 'wgs84Lon']]

solution_df

### 2) 응급의료기관별 기본정보 조회 - 응급실수, 수술실 수 확인

 * <b>기관ID</b>를 기준으로 응급실, 수술실 정보 등 응급의료기관 기본 정보 조회 가능
 * 응급실수 (hperyn)와 수술실 수(hpopyn) 정보를 확인

In [ ]:
# 대구광역시내이 응급의료기관 목록정보 조회하기
# url : https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytBassInfoInqire
# 인증키 지정 : serviceKey = key
# 응급의료기관 지정 : HPID = hpid
# 응급의료기관 하나씩 조회해야 함

# 응급의료기관에 대한 hpid값을 list로 추출
hpidList = list(solution_df['hpid'])

# 각 응급의료기관의 응급실 병상수(hperyn), 수술실 수(hpopyn) 를 저장하기 위한 빈 리스트 생성
hperyn_list = []
hpopyn_list = []

# 대구광역시의 응급의료기관을 hpid 기준으로 하나씩 조회하여, 응급실 병상수/수술실 수를 확인
# for문을 이용하여 hpid 하나씩 조회하고, hperyn, hpopyn 정보를 각 hperynList, hpopynList 리스트에 저장

for hpid in hpidList:
    url_basic = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytBassInfoInqire?serviceKey=' + key + '&HPID=' + hpid
    result = urlopen(url_basic, context = context)
    df_temp = pd.read_xml(result, xpath = './/item')
    
    if 'hperyn' in list(df_temp.columns): # df_temp에 hperyn 정보 없으면 0 append
        hperyn_list.append(df_temp['hperyn'][0])
    else:
        hperyn_list.append(0)
    
    if 'hpopyn' in list(df_temp.columns): # df_temp에 hpopyn 정보 없으면 0 append
        hpopyn_list.append(df_temp['hpopyn'][0])
    else:
        hpopyn_list.append(0)
    
hperyn_list, hpopyn_list

In [187]:
# 기존 solution_df에 응급실수, 수술실 수 정보 추가하기
# 응급실 수 : solution_df['hperyn']
# 수술실 수 : solution_df['hpopyn']
solution_df['hperyn'] = hperyn_list
solution_df['hpopyn'] = hpopyn_list

# 확인하기
solution_df

C:\Users\user\AppData\Local\Temp\ipykernel_21740\221897925.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solution_df['hperyn'] = hperyn_list
C:\Users\user\AppData\Local\Temp\ipykernel_21740\221897925.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solution_df['hpopyn'] = hpopyn_list


,hpid,dutyName,dutyAddr,dutyTel3,wgs84Lat,wgs84Lon,hperyn,hpopyn
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16
3,A1303386,계명대학교대구동산병원,"대구광역시 중구 달성로 56, 계명대학교대구동산병원 (동산동)",053-250-8481,35.869460,128.582931,0,0
4,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24
5,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4
6,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5
7,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18
8,A1300005,대구가톨릭대학교칠곡가톨릭병원,대구광역시 북구 칠곡중앙대로 440 (읍내동),053-320-2111,35.934880,128.549252,10,3
9,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2


In [188]:
## hperyn, hpopyn값이 0 이하인 병원 삭제
# 21 -> 18 : 3개 행 삭제
solution_df = solution_df[solution_df['hperyn'] > 0]
solution_df = solution_df[solution_df['hpopyn'] > 0]

In [189]:
## solution_df 크기 확인 : shape
solution_df.shape 


(18, 8)

In [190]:
## solution_df 를 파일로 저장 
## 파일명 : daegu_hospital_list.csv, index = False
solution_df.to_csv('daegu_hospital_list.csv', index = False)


In [191]:
## 정상적으로 저장되었는지 확인 
df_1 = pd.read_csv('daegu_hospital_list.csv')

df_1

,hpid,dutyName,dutyAddr,dutyTel3,wgs84Lat,wgs84Lon,hperyn,hpopyn
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18
7,A1300005,대구가톨릭대학교칠곡가톨릭병원,대구광역시 북구 칠곡중앙대로 440 (읍내동),053-320-2111,35.934880,128.549252,10,3
8,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2
9,A1300007,대구의료원,대구광역시 서구 평리로 157 (중리동),053-560-7288,35.859623,128.540561,10,4


### 3) 응급의료기관별 실시간 가용 병상 조회

 * 주소(대구광역시)를 기준으로 응급실 실시간 가용병상정보 등을 조회
 * 실시간 응급실(hvec), 수술실(hvoc) 가용 수량 정보 확인

In [192]:
# 응급실 실시간 가용병상 조회
# url : https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire
# 인증키 지정 : serviceKey = key
# 도시 지정 : STAGE1=city
# 출력 행수 : numofRows=100 
# page 번호 : pageNo = 1
# emrRealtime_big 변수명으로 저장

city = quote('대구광역시')

url = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire?serviceKey=' + key + '&STAGE1=' + city + '&pageNo=1&numOfRows=100'

result = urlopen(url, context = context)
emrRealtime_big = pd.read_xml(result, xpath = './/item')

In [193]:
## 응급실 실시간 가용병상 정보에서 기관코드(hpid), 응급실 병상수('hvec'), 수술실 수('hvoc') 정보만 추출하여 emRealtime_small 변수에 저장
## emrRealtime_big 중 [hpid, hvec, hvoc] 컬럼 활용

emRealtime_small = emrRealtime_big[['hpid', 'hvec', 'hvoc']]

emRealtime_small

,hpid,hvec,hvoc
0,A1300034,8,3
1,A1300004,9,9
2,A1300076,8,2
3,A1300001,6,14
4,A1300006,7,5
5,A1300003,-9,5
6,A1300110,3,4
7,A1300011,7,6
8,A1300045,1,0
9,A1300038,2,4


In [194]:
solution_df

,hpid,dutyName,dutyAddr,dutyTel3,wgs84Lat,wgs84Lon,hperyn,hpopyn
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16
4,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24
5,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4
6,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5
7,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18
8,A1300005,대구가톨릭대학교칠곡가톨릭병원,대구광역시 북구 칠곡중앙대로 440 (읍내동),053-320-2111,35.934880,128.549252,10,3
9,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2
10,A1300007,대구의료원,대구광역시 서구 평리로 157 (중리동),053-560-7288,35.859623,128.540561,10,4


In [195]:
# solution_df와 emrRealtime_small 데이터프레임을 결합하여 solution_df에 저장
# 결합 : merge 함수 활용

solution_df = pd.merge(solution_df, emRealtime_small)

solution_df

,hpid,dutyName,dutyAddr,dutyTel3,wgs84Lat,wgs84Lon,hperyn,hpopyn,hvec,hvoc
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,3
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4,15,5
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,-6,8
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24,-9,5
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,3
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5,2,4
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,9,9
7,A1300005,대구가톨릭대학교칠곡가톨릭병원,대구광역시 북구 칠곡중앙대로 440 (읍내동),053-320-2111,35.934880,128.549252,10,3,6,1
8,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2,8,2
9,A1300007,대구의료원,대구광역시 서구 평리로 157 (중리동),053-560-7288,35.859623,128.540561,10,4,16,4


In [196]:
solution_df.shape

(18, 10)

### 4) 응급의료기관별 중증질환자 수용 가능 여부 조회

 * 주소(대구광역시)를 기준으로 중증질환자 수용 가능 여부를 조회 (실시간 정보)

In [197]:
# 실시간 중증질환자 수용 가능 병원 조회
# url : https://apis.data.go.kr/B552657/ErmctInfoInqireService/getSrsillDissAceptncPosblInfoInqire
# 인증키 지정 : serviceKey = key
# 도시 지정 : STAGE1=city
# 출력 행수 : numOfRows=100 
# page 번호 : pageNo = 1

city = quote('대구광역시')

url = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getSrsillDissAceptncPosblInfoInqire?serviceKey=' + key + '&STAGE1=' + city + '&pageNo=1&numOfRows=100'
result = urlopen(url, context=context)
emrAcpt_big = pd.read_xml(result, xpath='.//item')

## 다른 API함수와 다르게 기관코드 컬럼명이 다름 (hpid --> dutyName)
## 기관코드 컬렴명을 'hpid'로 일치화시키기 위해, 컬럼명을 변경함

emrAcpt_big = emrAcpt_big.rename(columns = {'dutyName' : 'hpid'})

emrAcpt_big.head()

,hpid,MKioskTy1,MKioskTy10,MKioskTy11,MKioskTy12,MKioskTy13,MKioskTy14,MKioskTy15,MKioskTy16,MKioskTy17,...,MKioskTy5,MKioskTy6,MKioskTy7,MKioskTy8,MKioskTy9,MKioskTy10Msg,MKioskTy12Msg,MKioskTy14Msg,MKioskTy15Msg,MKioskTy27Msg
0,A1300034,정보미제공,Y,Y,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,...,정보미제공,정보미제공,Y,Y,Y,None,None,None,None,None
1,A1300004,Y,정보미제공,Y,정보미제공,Y,정보미제공,Y,Y,Y,...,Y,Y,Y,Y,Y,21일 이상 출생아부터 가능,0세부터 가능,.,.,.
2,A1300076,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,...,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,None,None,None,None,None
3,A1300001,Y,정보미제공,Y,정보미제공,Y,정보미제공,Y,Y,Y,...,Y,Y,정보미제공,Y,정보미제공,전체 연령 가능,평일 외래만 가능,None,전체 체중아 가능,전체 연령 가능
4,A1300006,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,...,정보미제공,정보미제공,Y,정보미제공,Y,None,None,None,None,None


In [198]:
## 실시간 중증질환자 수용 가능 병원정보에서 필요한 정보만 추출하여 emrAcpt_small 변수에 저장
## emrAcpt 중 [hpid, MKioskTy1, MKioskTy2, MKioskTy3, MKioskTy4, MKioskTy5, MKioskTy7,MKioskTy8, MKioskTy10, MKioskTy11] 컬럼 확인
# 'MKioskTy1' : '뇌출혈수술', 
# 'MKioskTy2' : '뇌경색의재관류',
# 'MKioskTy3' : '심근경색의재관류',
# 'MKioskTy4' : '복부손상의수술',
# 'MKioskTy5' : '사지접합의수술',
# 'MKioskTy7' : '응급투석',
# 'MKioskTy8' : '조산산모',
# 'MKioskTy10' : '신생아',
# 'MKioskTy11' : '중증화상'

emrAcpt_small = emrAcpt_big[['hpid', 'MKioskTy1', 'MKioskTy2', 'MKioskTy3', 'MKioskTy4', 'MKioskTy5', 'MKioskTy7','MKioskTy8', 'MKioskTy10', 'MKioskTy11']]
emrAcpt_small.head()

,hpid,MKioskTy1,MKioskTy2,MKioskTy3,MKioskTy4,MKioskTy5,MKioskTy7,MKioskTy8,MKioskTy10,MKioskTy11
0,A1300034,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,Y,Y,Y,Y
1,A1300004,Y,Y,Y,Y,Y,Y,Y,정보미제공,Y
2,A1300076,정보미제공,Y,Y,Y,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공
3,A1300001,Y,Y,Y,불가능,Y,정보미제공,Y,정보미제공,Y
4,A1300006,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,Y,정보미제공,정보미제공,정보미제공


In [199]:
# solution_df와 emrAcpt_small 데이터프레임을 결합하여 solution_df에 저장
# 결합 : merge 함수 활용

solution_df = pd.merge(solution_df, emrAcpt_small)

solution_df.head()
 

,hpid,dutyName,dutyAddr,dutyTel3,wgs84Lat,wgs84Lon,hperyn,hpopyn,hvec,hvoc,MKioskTy1,MKioskTy2,MKioskTy3,MKioskTy4,MKioskTy5,MKioskTy7,MKioskTy8,MKioskTy10,MKioskTy11
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,3,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,Y,정보미제공,정보미제공,Y
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4,15,5,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,-6,8,Y,Y,Y,Y,Y,Y,Y,Y,Y
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24,-9,5,Y,Y,Y,Y,Y,Y,정보미제공,Y,Y
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,3,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,Y,Y,정보미제공,Y


### 5) 자료 정비하기

 * 컬럼명 한글로 변경하기
 * 데이터 중 '정보 미제공', '불가능'은 'N'로 변경
 * 응급실/수술실 수가 0 이하의 수치가 나오는 비정상 사례 존재 --> 0이하의 수치는 모두 0으로 대체하여 정비
 * 응급실 가용율 (응급실 가용 수량 / 응급실수)을 계산
 * 응급실 포화도는 응급실 가용율에 따라 [불가/혼잡/보통/원활] 4단계로 구분

In [200]:
## solution_df의 컬럼명 확인

solution_df.columns

Index(['hpid', 'dutyName', 'dutyAddr', 'dutyTel3', 'wgs84Lat', 'wgs84Lon',
       'hperyn', 'hpopyn', 'hvec', 'hvoc', 'MKioskTy1', 'MKioskTy2',
       'MKioskTy3', 'MKioskTy4', 'MKioskTy5', 'MKioskTy7', 'MKioskTy8',
       'MKioskTy10', 'MKioskTy11'],
      dtype='object')

In [201]:
# column명을 한글로 교체하기 위해 딕셔너리로 정의

column_change = {  'hpid' : '병원코드',
                   'dutyName' : '병원명',
                   'dutyAddr' : '주소',
                   'dutyTel3' : '응급연락처',
                   'wgs84Lat' : '위도', 
                   'wgs84Lon' : '경도',
                   'hperyn' : '응급실수',
                   'hpopyn' : '수술실수',
                   'hvec' : '가용응급실수',
                   'hvoc' : '가용수술실수',
                   'MKioskTy1' : '뇌출혈', 
                   'MKioskTy2' : '뇌경색',
                   'MKioskTy3' : '심근경색',
                   'MKioskTy4' : '복부손상',
                   'MKioskTy5' : '사지접합',
                   'MKioskTy7' : '응급투석',
                   'MKioskTy8' : '조산산모',
                   'MKioskTy10' : '신생아',
                   'MKioskTy11' : '중증화상',
                   
                   }

In [202]:
# solution_df의 column명 변경하기 : rename
solution_df = solution_df.rename(columns = column_change)

solution_df.head()

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,뇌출혈,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,3,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,Y,정보미제공,정보미제공,Y
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4,15,5,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,-6,8,Y,Y,Y,Y,Y,Y,Y,Y,Y
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24,-9,5,Y,Y,Y,Y,Y,Y,정보미제공,Y,Y
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,3,정보미제공,정보미제공,정보미제공,정보미제공,정보미제공,Y,Y,정보미제공,Y


In [203]:
# 중증질환 수용 가능 여부 데이터 중 정보 미제공, 불가능은 N로 변경 : replace
solution_df = solution_df.replace(['정보미제공','불가능'], 'N')
 
solution_df.head()

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,뇌출혈,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,3,N,N,N,N,N,Y,N,N,Y
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4,15,5,N,N,N,N,N,N,N,N,N
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,-6,8,Y,Y,Y,Y,Y,Y,Y,Y,Y
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24,-9,5,Y,Y,Y,Y,Y,Y,N,Y,Y
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,3,N,N,N,N,N,Y,Y,N,Y


In [204]:
solution_df

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,뇌출혈,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,3,N,N,N,N,N,Y,N,N,Y
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4,15,5,N,N,N,N,N,N,N,N,N
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,-6,8,Y,Y,Y,Y,Y,Y,Y,Y,Y
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24,-9,5,Y,Y,Y,Y,Y,Y,N,Y,Y
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,3,N,N,N,N,N,Y,Y,N,Y
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5,2,4,Y,Y,Y,Y,N,Y,Y,N,Y
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,9,9,Y,Y,Y,Y,Y,Y,Y,N,Y
7,A1300005,대구가톨릭대학교칠곡가톨릭병원,대구광역시 북구 칠곡중앙대로 440 (읍내동),053-320-2111,35.934880,128.549252,10,3,6,1,N,N,N,N,N,N,N,N,N
8,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2,8,2,N,Y,Y,Y,N,N,N,N,N
9,A1300007,대구의료원,대구광역시 서구 평리로 157 (중리동),053-560-7288,35.859623,128.540561,10,4,16,4,N,N,N,N,N,N,N,N,Y


In [205]:
solution_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   병원코드    18 non-null     object 
 1   병원명     18 non-null     object 
 2   주소      18 non-null     object 
 3   응급연락처   18 non-null     object 
 4   위도      18 non-null     float64
 5   경도      18 non-null     float64
 6   응급실수    18 non-null     int64  
 7   수술실수    18 non-null     int64  
 8   가용응급실수  18 non-null     int64  
 9   가용수술실수  18 non-null     int64  
 10  뇌출혈     18 non-null     object 
 11  뇌경색     18 non-null     object 
 12  심근경색    18 non-null     object 
 13  복부손상    18 non-null     object 
 14  사지접합    18 non-null     object 
 15  응급투석    18 non-null     object 
 16  조산산모    18 non-null     object 
 17  신생아     18 non-null     object 
 18  중증화상    18 non-null     object 
dtypes: float64(2), int64(4), object(13)
memory usage: 2.8+ KB


In [206]:
solution_df.loc[solution_df['응급실수'] < 0, '응급실수'] = 0

solution_df

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,뇌출혈,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,3,N,N,N,N,N,Y,N,N,Y
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4,15,5,N,N,N,N,N,N,N,N,N
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,-6,8,Y,Y,Y,Y,Y,Y,Y,Y,Y
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24,-9,5,Y,Y,Y,Y,Y,Y,N,Y,Y
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,3,N,N,N,N,N,Y,Y,N,Y
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5,2,4,Y,Y,Y,Y,N,Y,Y,N,Y
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,9,9,Y,Y,Y,Y,Y,Y,Y,N,Y
7,A1300005,대구가톨릭대학교칠곡가톨릭병원,대구광역시 북구 칠곡중앙대로 440 (읍내동),053-320-2111,35.934880,128.549252,10,3,6,1,N,N,N,N,N,N,N,N,N
8,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2,8,2,N,Y,Y,Y,N,N,N,N,N
9,A1300007,대구의료원,대구광역시 서구 평리로 157 (중리동),053-560-7288,35.859623,128.540561,10,4,16,4,N,N,N,N,N,N,N,N,Y


In [207]:
## 응급실수/가용응급실수, 수술실수/가용수술실 수가 0보다 작은 경우는 비정상 데이터로 추정
## 0보다 작은 수는 0으로 변경
# loc 활용

solution_df.loc[solution_df['응급실수'] < 0, '응급실수'] = 0
solution_df.loc[solution_df['가용응급실수'] < 0, '가용응급실수'] = 0
solution_df.loc[solution_df['수술실수'] < 0, '수술실수'] = 0
solution_df.loc[solution_df['가용수술실수'] < 0, '가용수술실수'] = 0

solution_df

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,뇌출혈,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,3,N,N,N,N,N,Y,N,N,Y
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4,15,5,N,N,N,N,N,N,N,N,N
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,0,8,Y,Y,Y,Y,Y,Y,Y,Y,Y
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24,0,5,Y,Y,Y,Y,Y,Y,N,Y,Y
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,3,N,N,N,N,N,Y,Y,N,Y
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5,2,4,Y,Y,Y,Y,N,Y,Y,N,Y
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,9,9,Y,Y,Y,Y,Y,Y,Y,N,Y
7,A1300005,대구가톨릭대학교칠곡가톨릭병원,대구광역시 북구 칠곡중앙대로 440 (읍내동),053-320-2111,35.934880,128.549252,10,3,6,1,N,N,N,N,N,N,N,N,N
8,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2,8,2,N,Y,Y,Y,N,N,N,N,N
9,A1300007,대구의료원,대구광역시 서구 평리로 157 (중리동),053-560-7288,35.859623,128.540561,10,4,16,4,N,N,N,N,N,N,N,N,Y


In [208]:
# 응급실 가용율을 구하여 새로운 컬럼으로 추가하기
# 컬렴명 : '응급실가용율'
# 산식 : 가용 응급실수 / 응급실 수
# 소수 둘째 자리까지 구하기 round() 활용

solution_df['응급실가용율'] = round(solution_df['가용응급실수'] / solution_df['응급실수'],2)

# 응급실 가용율이 1이 넘는 경우는 1로 대체
# loc 활용
solution_df.loc[solution_df['응급실가용율'] > 1, '응급실가용율'] = 1

solution_df

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,뇌출혈,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,3,N,N,N,N,N,Y,N,N,Y,0.70
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4,15,5,N,N,N,N,N,N,N,N,N,1.00
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,0,8,Y,Y,Y,Y,Y,Y,Y,Y,Y,0.00
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24,0,5,Y,Y,Y,Y,Y,Y,N,Y,Y,0.00
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,3,N,N,N,N,N,Y,Y,N,Y,0.70
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5,2,4,Y,Y,Y,Y,N,Y,Y,N,Y,0.18
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,9,9,Y,Y,Y,Y,Y,Y,Y,N,Y,0.38
7,A1300005,대구가톨릭대학교칠곡가톨릭병원,대구광역시 북구 칠곡중앙대로 440 (읍내동),053-320-2111,35.934880,128.549252,10,3,6,1,N,N,N,N,N,N,N,N,N,0.60
8,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2,8,2,N,Y,Y,Y,N,N,N,N,N,1.00
9,A1300007,대구의료원,대구광역시 서구 평리로 157 (중리동),053-560-7288,35.859623,128.540561,10,4,16,4,N,N,N,N,N,N,N,N,Y,1.00


In [213]:
# 응급실 가용율에 따라 포화도 분류
# 응급실 가용율 구분 단계 : ~0.1, 0.1 ~ 0.3, 0.3 ~ 0.6, 0.6 ~ 
# 포화도 명칭 : ['불가', '혼잡', '보통', '원활']
# pd.cut() 활용

bin = [-np.inf, 0.1, 0.3, 0.6, np.inf]
label = ['불가', '혼잡', '보통', '원활']

solution_df['응급실포화도'] =  pd.cut(solution_df['응급실가용율'], bins = bin, labels = label)

solution_df.head()

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,3,...,N,N,N,N,Y,N,N,Y,0.7,원활
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4,15,5,...,N,N,N,N,N,N,N,N,1.0,원활
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,0,8,...,Y,Y,Y,Y,Y,Y,Y,Y,0.0,불가
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24,0,5,...,Y,Y,Y,Y,Y,N,Y,Y,0.0,불가
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,3,...,N,N,N,N,Y,Y,N,Y,0.7,원활


### 6) 상황에 따른 병원 데이터 출력하기

 * 상황1) 응급실 수용이 가능한 병원
 * 상황2) 중증질환자 수용이 가능하고 응급실 수용이 가능한 병원

In [216]:
# 상황1) 응급실 수용이 가능한 병원
# 조건1 : 응급실 포화도가 불가가 아닌 병원을 출력

solution_df_1 = solution_df.loc[solution_df['응급실포화도'] != '불가', :]

solution_df_1

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,3,...,N,N,N,N,Y,N,N,Y,0.70,원활
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4,15,5,...,N,N,N,N,N,N,N,N,1.00,원활
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,3,...,N,N,N,N,Y,Y,N,Y,0.70,원활
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5,2,4,...,Y,Y,Y,N,Y,Y,N,Y,0.18,혼잡
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,9,9,...,Y,Y,Y,Y,Y,Y,N,Y,0.38,보통
7,A1300005,대구가톨릭대학교칠곡가톨릭병원,대구광역시 북구 칠곡중앙대로 440 (읍내동),053-320-2111,35.934880,128.549252,10,3,6,1,...,N,N,N,N,N,N,N,N,0.60,보통
8,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2,8,2,...,Y,Y,Y,N,N,N,N,N,1.00,원활
9,A1300007,대구의료원,대구광역시 서구 평리로 157 (중리동),053-560-7288,35.859623,128.540561,10,4,16,4,...,N,N,N,N,N,N,N,Y,1.00,원활
11,A1300110,더블유병원,대구광역시 달서구 달구벌대로 1632 (감삼동),053-550-5019,35.852642,128.544969,10,8,3,4,...,N,N,N,N,N,N,N,N,0.30,혼잡
12,A1300034,드림종합병원,대구광역시 남구 대명로 153 (대명동),053-640-8575,35.839635,128.574355,10,3,8,3,...,N,N,N,N,Y,Y,Y,Y,0.80,원활


In [218]:
# 상황2) 중증질환자 수용이 가능하고, 응급실 수용이 가능한 병원
# 조건1 : '뇌출혈' 중증질환자 수용 및 수술실 수용 가능한(가용 수술실수가 1이상) 응급의료기관 조회
# 조건2 : 응급실 포화도가 불가가 아닌 병원을 출력

special_m = '뇌출혈'

solution_df_1 = solution_df_1.loc[(solution_df_1[special_m] == 'Y') & (solution_df_1['가용수술실수'] >= 1), :]

solution_df_1

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,뇌경색,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5,2,4,...,Y,Y,Y,N,Y,Y,N,Y,0.18,혼잡
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,9,9,...,Y,Y,Y,Y,Y,Y,N,Y,0.38,보통
14,A1300001,영남대학교병원,대구광역시 남구 현충로 170 (대명동),053-620-3177,35.847506,128.585131,30,19,6,14,...,Y,Y,N,Y,N,Y,N,Y,0.20,혼잡


### 7) 병원 거리 구하기

 * 병원 위치(위도/경도)와 환자의 위치(임의의 위도/경도) 간의 거리 측정
 * haversine 라이브러리를 활용 (https://pypi.org/project/haversine/)
 * 병원 ~ 환자간의 거리를 ['2km이내', '5km이내', '10km이내', '10km이상'] 4단계로 구분

In [37]:
# 환자와 병원간의 거리 측정을 위한 라이브러리 설치
!pip install haversine

In [40]:
# 라이브러리 import
from haversine import haversine

# 거리 측정 예시
# 위치1 : (35.209710, 129.004751)
# 위치2 : (35.1631139, 129.1635509)
loc1 = (35.209710, 129.004751)
loc2 = (35.1631139, 129.1635509)

haversine(loc1, loc2, 'km') # 두 위치 간 'km'단위로 거리측정

15.333288646032319

In [233]:
# 환자 위치 : 서문시장 내
patient= (35.8690760, 128.5811616)  #Latitude, Longitude

# 환자와 병원간 거리를 저장할 빈 리스트 만들기 
# 변수명 : distance

distance = []

# solution_df의 복사본을 만들어 distance_df에 저장하기

distance_df = solution_df.copy()

for idx, row in distance_df.iterrows():
    distance.append(round(haversine(patient, (row['위도'], row['경도']), 'km'),2))# haversine 함수를 이용하여 환자와 병원간 거리 구하기 
    # 단위 : km
    # 소수 둘째자리까지 구하기
    # distance 변수에 하나씩 채우기
  

# distance_df의 '거리' 컬럼에, distance 값 저장하기
distance_df['거리'] = distance

# 확인하기
distance_df

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,심근경색,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도,거리
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,3,...,N,N,N,Y,N,N,Y,0.70,원활,11.68
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4,15,5,...,N,N,N,N,N,N,N,1.00,원활,7.57
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,0,8,...,Y,Y,Y,Y,Y,Y,Y,0.00,불가,2.11
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24,0,5,...,Y,Y,Y,Y,N,Y,Y,0.00,불가,9.26
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,3,...,N,N,N,Y,Y,N,Y,0.70,원활,0.71
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5,2,4,...,Y,Y,N,Y,Y,N,Y,0.18,혼잡,7.88
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,9,9,...,Y,Y,Y,Y,Y,N,Y,0.38,보통,3.15
7,A1300005,대구가톨릭대학교칠곡가톨릭병원,대구광역시 북구 칠곡중앙대로 440 (읍내동),053-320-2111,35.934880,128.549252,10,3,6,1,...,N,N,N,N,N,N,N,0.60,보통,7.86
8,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2,8,2,...,Y,Y,N,N,N,N,N,1.00,원활,1.90
9,A1300007,대구의료원,대구광역시 서구 평리로 157 (중리동),053-560-7288,35.859623,128.540561,10,4,16,4,...,N,N,N,N,N,N,Y,1.00,원활,3.81


In [237]:
# distance_df의 '거리'에 따라 거리구분 분류 : cut()
# '거리구분' 명칭 : ['2km이내', '5km이내', '10km이내', '10km이상']

bin = [-np.inf, 2, 5, 10, np.inf]
label = ['2km이내', '5km이내', '10km이내', '10km이상']

distance_df['거리구분'] =  pd.cut(distance_df['거리'], bins = bin, labels = label)

distance_df.head()

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도,거리,거리구분
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,3,...,N,N,Y,N,N,Y,0.7,원활,11.68,10km이상
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4,15,5,...,N,N,N,N,N,N,1.0,원활,7.57,10km이내
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,0,8,...,Y,Y,Y,Y,Y,Y,0.0,불가,2.11,5km이내
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24,0,5,...,Y,Y,Y,N,Y,Y,0.0,불가,9.26,10km이내
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,3,...,N,N,Y,Y,N,Y,0.7,원활,0.71,2km이내


In [243]:
# 가까운 거리순으로 : '거리구분
# 응급실이 원활한 순서로 : '응급실포화도'
# 정렬하기 : sort_values

distance_df.sort_values(['거리구분', '응급실포화도'], ascending = [True, False])

,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도,거리,거리구분
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,3,...,N,N,Y,Y,N,Y,0.70,원활,0.71,2km이내
8,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2,8,2,...,Y,N,N,N,N,N,1.00,원활,1.90,2km이내
9,A1300007,대구의료원,대구광역시 서구 평리로 157 (중리동),053-560-7288,35.859623,128.540561,10,4,16,4,...,N,N,N,N,N,Y,1.00,원활,3.81,5km이내
12,A1300034,드림종합병원,대구광역시 남구 대명로 153 (대명동),053-640-8575,35.839635,128.574355,10,3,8,3,...,N,N,Y,Y,Y,Y,0.80,원활,3.33,5km이내
15,A1300006,의료법인구의료재단구병원,대구광역시 달서구 감삼북길 141 (감삼동),053-557-7119,35.850823,128.541483,11,8,7,5,...,N,N,Y,N,N,N,0.64,원활,4.11,5km이내
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,9,9,...,Y,Y,Y,Y,N,Y,0.38,보통,3.15,5km이내
11,A1300110,더블유병원,대구광역시 달서구 달구벌대로 1632 (감삼동),053-550-5019,35.852642,128.544969,10,8,3,4,...,N,N,N,N,N,N,0.30,혼잡,3.74,5km이내
14,A1300001,영남대학교병원,대구광역시 남구 현충로 170 (대명동),053-620-3177,35.847506,128.585131,30,19,6,14,...,N,Y,N,Y,N,Y,0.20,혼잡,2.43,5km이내
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16,0,8,...,Y,Y,Y,Y,Y,Y,0.00,불가,2.11,5km이내
10,A1300009,대구파티마병원,대구광역시 동구 아양로 99 (신암동),053-940-7119,35.883934,128.623835,28,13,0,4,...,Y,N,Y,Y,Y,Y,0.00,불가,4.18,5km이내


### 8) 중증 질환자를 수용 가능한 응급의료기관을 가까운 거리순으로 출력

In [246]:
# 중증질환자 수용이 가능하고, 응급실 수용이 가능한 병원 중 환자 위치에서 가까운 병원
# 조건1 : '중증화상' 중증질환자 수용 및 수술실 수용 가능한 응급으료기관 조회
# 조건2 :응급실 포화도가 불가가 아닌 병원
# 정렬 : 환자 위치를 중심으로 가장 가까운 병원부터 출력

# 환자 위치 : 서문시장 내
patient= (35.8690760, 128.5811616)  #Latitude, Longitude

# 조건1 : '중증화상' 중증질환자 수용 및 수술실 수용 가능한 응급으료기관 조회
special_m = '중증화상'

distance_df = distance_df.loc[(distance_df['중증화상'] == 'Y') & (distance_df['가용수술실수'] >= 1), :]

# 조건2 :응급실 포화도가 불가가 아닌 병원
distance_df = distance_df.loc[distance_df['응급실포화도'] != '불가',:]

# 조건1, 2에 해당되는 응급의료기관 정보를 distance_df에 저장하기
distance_df



# 환자와 병원간 거리를 저장할 빈 리스트 만들기 
# 변수명 : distance
distance = []


for idx, row in distance_df.iterrows():
    distance.append(round(haversine(patient, (row['위도'], row['경도']), 'km'),2))# haversine 함수를 이용하여 환자와 병원간 거리 구하기 
    # 단위 : km
    # 소수 둘째자리까지 구하기
    # distance 변수에 하나씩 채우기
    

#distance_df의 '거리' 컬럼에, distance 값 저장하기
distance_df['거리'] = distance

# distancd_df의 '거리'에 따라 거리구분 분류 : cut()
# '거리구분' 명칭 : ['2km이내', '5km이내', '10km이내', '10km이상']
bin = [-np.inf, 2, 5, 10, np.inf]
label = ['2km이내', '5km이내', '10km이내', '10km이상']

distance_df['거리구분'] =  pd.cut(distance_df['거리'], bins = bin, labels = label)


# 환자 위치에서 가까운 거리순으로, 응급실 포화도가 원활한 순서로 정렬하기
distance_df.sort_values(['거리구분', '응급실포화도'], ascending = [True, False])


,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도,거리,거리구분
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,7,3,...,N,N,Y,Y,N,Y,0.70,원활,0.71,2km이내
9,A1300007,대구의료원,대구광역시 서구 평리로 157 (중리동),053-560-7288,35.859623,128.540561,10,4,16,4,...,N,N,N,N,N,Y,1.00,원활,3.81,5km이내
12,A1300034,드림종합병원,대구광역시 남구 대명로 153 (대명동),053-640-8575,35.839635,128.574355,10,3,8,3,...,N,N,Y,Y,Y,Y,0.80,원활,3.33,5km이내
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,9,9,...,Y,Y,Y,Y,N,Y,0.38,보통,3.15,5km이내
14,A1300001,영남대학교병원,대구광역시 남구 현충로 170 (대명동),053-620-3177,35.847506,128.585131,30,19,6,14,...,N,Y,N,Y,N,Y,0.20,혼잡,2.43,5km이내
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5,2,4,...,Y,N,Y,Y,N,Y,0.18,혼잡,7.88,10km이내
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,7,3,...,N,N,Y,N,N,Y,0.70,원활,11.68,10km이상


### 9) 특정 중증질환 및 환자 위치에 따른 병원 실시간 정보 및 거리를 구하는 함수 작성하기

 * 작성 방법
    * 1) 함수선언하기
    * 2) 병원 리스트 csv 파일 불러오기
    * 3) 병원 실시간 정보 가져오기
    * 4) 병원 응급실 가용율, 포화도 계산하기
    * 5) 환자 수용 가능한 병원 구하기 
    * 6) 환자와 병원간 거리 구하기
    * 7) 거리 구간 구하기
    * 8) 결과값 반환하기



In [247]:
## tabulate library가 없는 경우 설치 필요
## pandas.to_markdown() 시 tabulate library가 필요함
# !pip install tabulate 

In [248]:
solution_df = pd.read_csv('daegu_hospital_list.csv')

solution_df

,hpid,dutyName,dutyAddr,dutyTel3,wgs84Lat,wgs84Lon,hperyn,hpopyn
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18
7,A1300005,대구가톨릭대학교칠곡가톨릭병원,대구광역시 북구 칠곡중앙대로 440 (읍내동),053-320-2111,35.934880,128.549252,10,3
8,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2
9,A1300007,대구의료원,대구광역시 서구 평리로 157 (중리동),053-560-7288,35.859623,128.540561,10,4


In [250]:
import pandas as pd
from urllib.parse import quote
from haversine import haversine
import ssl
from urllib.request import urlopen
import numpy as np

context=ssl.create_default_context()
context.set_ciphers("DEFAULT")

#########################################
# 1) 함수 선언하기                       #
#########################################
# 함수명 : find_hospital
# 매개변수 : special_m (중증질환명), lati, long (환자 위치)

def find_hospital(special_m, lati, long):
    #  [국립중앙의료원 - 전국응급의료기관 조회 서비스] 활용을 위한 개인 일반 인증키(Encoding) 저장

    key = "dmxRDNK9srMd2YgqgrNr94Qwm66GQ2qOeaR52ewsjheCoPNcAzknauVpluJX/EIgKY0Vo85YDDvCD23VN6MdvQ=="

    # city = 대구광역시, 인코딩 필요
    city = quote('대구광역시')

    ############################################################
    # 2) 병원 리스트 csv 파일 불러오기 (daegu_hospital_list.csv) #
    ############################################################
    solution_df = pd.read_csv('daegu_hospital_list.csv')

    ############################################################
    # 3) 병원 실시간 정보 가져오기                               #
    ############################################################

    # 응급실 실시간 가용병상 
    url = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire' + '?serviceKey=' + key + '&STAGE1=' + city + '&pageNo=1&numOfRows=100'

    result = urlopen(url, context = context)
    emrRealtime_big = pd.read_xml(result, xpath='.//item')

    ## 응급실 실시간 가용병상 정보에서 기관코드(hpid), 응급실 병상수('hvec'), 수술실 수('hvoc') 정보만 추출하여 emRealtime_small 변수에 저장
    ## emrRealtime_big 중 [hpid, hvec, hvoc] 컬럼 활용
    emrRealtime_small = emrRealtime_big[['hpid', 'hvec', 'hvoc']]
    
    # solution_df와 emrRealtime_small 데이터프레임을 결합하여 solution_df에 저장
    solution_df = pd.merge(solution_df, emrRealtime_small)
    
    # 실시간 중증질환자 수용 가능 병원 조회
    url_acpt = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getSrsillDissAceptncPosblInfoInqire?serviceKey=' + key + '&STAGE1=' + city + '&pageNo=1&numOfRows=100'

    result = urlopen(url_acpt, context=context)
    emrAcpt_big = pd.read_xml(result, xpath='.//item')

    ## 다른 API함수와 다르게 기관코드 컬럼명이 다름 (hpid --> dutyName)
    ## 기관코드 컬렴명을 'hpid'로 일치화시키기 위해, 컬럼명을 변경함
    emrAcpt_big = emrAcpt_big.rename(columns = {'dutyName' : 'hpid'})
    
    ## 실시간 중증질환자 수용 가능 병원정보에서 필요한 정보만 추출하여 emrAcpt_small 변수에 저장
    ## emrAcpt 중 [hpid, MKioskTy1, MKioskTy2, MKioskTy3, MKioskTy4, MKioskTy5, MKioskTy7,MKioskTy8, MKioskTy10, MKioskTy11] 컬럼 확인
    emrAcpt_small = emrAcpt_big[['hpid', 'MKioskTy1', 'MKioskTy2', 'MKioskTy3', 'MKioskTy4', 'MKioskTy5', 'MKioskTy7','MKioskTy8', 'MKioskTy10', 'MKioskTy11']]


    # solution_df와 emrRealtime_small 데이터프레임을 결합하여 solution_df에 저장
    solution_df = pd.merge(solution_df, emrAcpt_small)
    
    ############################################################
    # 4) 자료 정비하기                                          #
    ############################################################

    # solution_df의 컬럼명 변경하기

    column_change = {  'hpid' : '병원코드',
                    'dutyName' : '병원명',
                    'dutyAddr' : '주소',
                    'dutyTel3' : '응급연락처',
                    'wgs84Lat' : '위도', 
                    'wgs84Lon' : '경도',
                    'hperyn' : '응급실수',
                    'hpopyn' : '수술실수',
                    'hvec' : '가용응급실수',
                    'hvoc' : '가용수술실수',
                    'MKioskTy1' : '뇌출혈', 
                    'MKioskTy2' : '뇌경색',
                    'MKioskTy3' : '심근경색',
                    'MKioskTy4' : '복부손상',
                    'MKioskTy5' : '사지접합',
                    'MKioskTy7' : '응급투석',
                    'MKioskTy8' : '조산산모',
                    'MKioskTy10' : '신생아',
                    'MKioskTy11' : '중증화상',                   
                    }

    solution_df = solution_df.rename(columns = column_change)
    
    # 중증질환 수용 가능 여부 데이터 중 정보 미제공, 불가능은 N로 변경 : replace
    solution_df = solution_df.replace(['정보미제공','불가능'], 'N')

    ## 응급실수/가용응급실수, 수술실수/가용수술실 수가 0보다 작은 경우는 비정상 데이터로 추정
    ## 0보다 작은 수는 0으로 변경
    solution_df.loc[solution_df['응급실수'] < 0, '응급실수'] = 0
    solution_df.loc[solution_df['가용응급실수'] < 0, '가용응급실수'] = 0
    solution_df.loc[solution_df['수술실수'] < 0, '수술실수'] = 0
    solution_df.loc[solution_df['가용수술실수'] < 0, '가용수술실수'] = 0

    # 응급실 가용율을 구하여 새로운 컬럼으로 추가하기
    # 컬렴명 : '응급실가용율'
    # 산식 : 가용 응급실수 / 응급실 수
    # 소수 둘째 자리까지 구하기 round() 활용
    solution_df['응급실가용율'] = round(solution_df['가용응급실수'] / solution_df['응급실수'],2)
    
    # 응급실 가용율이 1이 넘는 경우는 1로 대체
    solution_df.loc[solution_df['응급실가용율'] > 1, '응급실가용율'] = 1
    

    # 응급실 가용율에 따라 포화도 분류
    # 응급실 가용율 구분 단계 : ~0.1, 0.1 ~ 0.3, 0.3 ~ 0.6, 0.6 ~ 
    # 포화도 명칭 : ['불가', '혼잡', '보통', '원활']
    # pd.cut() 활용
    bin = [-np.inf, 0.1, 0.3, 0.6, np.inf]
    label = ['불가', '혼잡', '보통', '원활']

    solution_df['응급실포화도'] =  pd.cut(solution_df['응급실가용율'], bins = bin, labels = label)

    ############################################################
    # 5) 환자 수용 가능한 병원 구하기                            #  
    ############################################################

    # 매개변수 special_m로 받은 중증질환이 중증질환 리스트에 포함될 경우
    # 중증질환 리스트 :  ['뇌출혈', '뇌경색', '심근경색', '복부손상', '사지접합', '응급투석', '조산산모', '신생아','중증화상' ]

    if special_m in ['뇌출혈', '뇌경색', '심근경색', '복부손상', '사지접합', '응급투석', '조산산모', '신생아','중증화상']:
        distance_df = solution_df.loc[(solution_df[special_m] == 'Y') & (solution_df['응급실포화도'] != '불가'), :]
        

    # 매개변수 special_m 값이 중증질환 리스트에 포함이 안되는 경우
    else :
        distance_df = solution_df.loc[solution_df['응급실포화도'] != '불가', :]
        
        

    ############################################################
    # 6) 환자와 병원간 거리 구하기                               # 
    ############################################################
    # 환자와 병원간 거리를 저장할 빈 리스트 만들기 
    # 변수명 : distance
    distance = []
    

    #매개변수 lati, long으로 환자의 위치 정보 받기
    # 변수명 : patient
    patient = (lati, long)

    for idx, row in distance_df.iterrows():
        distance.append(round(haversine(patient, (row['위도'], row['경도']), 'km'),2))
        

    #distance_df의 '거리' 컬럼에, distance 값 저장하기
    distance_df['거리'] = distance

    ############################################################
    # 7) 거리 구간 구하기                                       #
    ############################################################
    # distancd_df의 '거리'에 따라 거리구분 분류 : cut()
    # '거리구분' 명칭 : ['2km이내', '5km이내', '10km이내', '10km이상']
    bin = [-np.inf, 2, 5, 10, np.inf]
    label = ['2km이내', '5km이내', '10km이내', '10km이상']

    distance_df['거리구분'] =  pd.cut(distance_df['거리'], bins = bin, labels = label)
    

    
    ############################################################
    # 8) 결과값 반환하기                                        #
    ############################################################
    return distance_df
    


In [264]:
# 중증환자 수용 가능한, 가까운 병원 조회 #1
# 환자 정보 
#   - 중증 질환 : 응급투석
#   - 환자 위치 : 대구역 근처 (35.8765167, 128.5972922))
# 거리순, 응급실 포화도 순으로 결과 출력하기 (pandas의 sort_values(), to_markdown() 활용)


display(find_hospital('응급투석', 35.8765167, 128.5972922).sort_values(['거리구분', '응급실포화도','거리'], ascending=[True, False, True]))
# print(find_hospital('응급투석', 35.8765167, 128.5972922).sort_values(['거리구분', '응급실포화도','거리'], ascending=[True, False, True]).to_markdown())

C:\Users\user\AppData\Local\Temp\ipykernel_21740\2594541295.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distance_df['거리'] = distance
C:\Users\user\AppData\Local\Temp\ipykernel_21740\2594541295.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distance_df['거리구분'] =  pd.cut(distance_df['거리'], bins = bin, labels = label)


,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도,거리,거리구분
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4,8,1,...,N,N,Y,Y,N,Y,0.80,원활,1.01,2km이내
12,A1300034,드림종합병원,대구광역시 남구 대명로 153 (대명동),053-640-8575,35.839635,128.574355,10,3,8,2,...,N,N,Y,Y,Y,Y,0.80,원활,4.59,5km이내
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,7,10,...,Y,Y,Y,Y,N,Y,0.29,혼잡,4.59,5km이내
15,A1300006,의료법인구의료재단구병원,대구광역시 달서구 감삼북길 141 (감삼동),053-557-7119,35.850823,128.541483,11,8,8,0,...,N,N,Y,N,N,N,0.73,원활,5.78,10km이내
13,A1300045,삼일병원,"대구광역시 달서구 월배로 436, 지하1,지상1~11층 (송현동)",053-659-3119,35.832748,128.553855,10,2,4,2,...,N,N,Y,N,Y,N,0.40,보통,6.25,10km이내
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5,6,4,...,Y,N,Y,Y,N,Y,0.55,보통,9.47,10km이내
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4,8,2,...,N,N,Y,N,N,Y,0.80,원활,10.57,10km이상


In [263]:
# 중증환자 수용 가능한, 가까운 병원 조회 #2
# 환자 정보 
#   - 중증 질환 : 심근경색
#   - 환자 위치 : 수성못 근처 (35.8259090, 128.616528))
# 거리순, 응급실 포화도 순으로 결과 출력하기 (pandas의 sort_values(), to_markdown() 활용)

display(find_hospital('심근경색', 35.8765167, 128.5972922).sort_values(['거리구분', '응급실포화도', '거리'], ascending=[True, False, True]))
# print(find_hospital('심근경색', 35.8765167, 128.5972922).sort_values(['거리구분', '응급실포화도', '거리'], ascending=[True, False, True]).to_markdown())

C:\Users\user\AppData\Local\Temp\ipykernel_21740\2594541295.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distance_df['거리'] = distance
C:\Users\user\AppData\Local\Temp\ipykernel_21740\2594541295.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distance_df['거리구분'] =  pd.cut(distance_df['거리'], bins = bin, labels = label)


,병원코드,병원명,주소,응급연락처,위도,경도,응급실수,수술실수,가용응급실수,가용수술실수,...,복부손상,사지접합,응급투석,조산산모,신생아,중증화상,응급실가용율,응급실포화도,거리,거리구분
8,A1300076,대구굿모닝병원,대구광역시 남구 성당로 224 (대명동),053-620-9119,35.852967,128.574067,7,2,9,2,...,Y,N,N,N,N,N,1.00,원활,3.35,5km이내
14,A1300001,영남대학교병원,대구광역시 남구 현충로 170 (대명동),053-620-3177,35.847506,128.585131,30,19,4,14,...,N,Y,N,Y,N,Y,0.13,혼잡,3.41,5km이내
6,A1300004,대구가톨릭대학교병원,대구광역시 남구 두류공원로17길 33 (대명동),053-650-3025,35.843066,128.567426,24,18,7,10,...,Y,Y,Y,Y,N,Y,0.29,혼잡,4.59,5km이내
5,A1300038,나사렛종합병원,"대구광역시 달서구 월배로 97, 나사렛병원 (진천동)",053-643-3119,35.814700,128.525095,11,5,6,4,...,Y,N,Y,Y,N,Y,0.55,보통,9.47,10km이내


# 추가실습

1. **haversine** 적용 최단 직선거리 -> 도로data 활용 최단 시간거리 분석
2. **KT 본사**(경기도 성남시) 지역으로 공공데이터 활용

In [284]:
# 개인 인증키 저장
# [국립중앙의료원 - 전국응급의료기관 조회 서비스] 활용을 위한 개인 일반 인증키(Encoding) 값 저장
key = "dmxRDNK9srMd2YgqgrNr94Qwm66GQ2qOeaR52ewsjheCoPNcAzknauVpluJX/EIgKY0Vo85YDDvCD23VN6MdvQ=="

# 조회할 도시 지정
# city = 대구광역시, 인코딩 필요
city = quote('경기도')

url = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire' + '?serviceKey=' + key + '&Q0=' + city + '&pageNo=1&numOfRows=100'

print(url) # 만든 url 출력

result = urlopen(url, context = context)
emrList = pd.read_xml(result, xpath='.//item')

solution_df = emrList[['hpid', 'dutyName', 'dutyAddr', 'dutyTel3', 'wgs84Lat', 'wgs84Lon']]

hpidList = list(solution_df['hpid'])

# 각 응급의료기관의 응급실 병상수(hperyn), 수술실 수(hpopyn) 를 저장하기 위한 빈 리스트 생성
hperyn_list = []
hpopyn_list = []

# 대구광역시의 응급의료기관을 hpid 기준으로 하나씩 조회하여, 응급실 병상수/수술실 수를 확인
# for문을 이용하여 hpid 하나씩 조회하고, hperyn, hpopyn 정보를 각 hperynList, hpopynList 리스트에 저장

for hpid in hpidList:
    url_basic = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytBassInfoInqire?serviceKey=' + key + '&HPID=' + hpid
    result = urlopen(url_basic, context = context)
    df_temp = pd.read_xml(result, xpath = './/item')
    
    if 'hperyn' in list(df_temp.columns): # df_temp에 hperyn 정보 없으면 0 append
        hperyn_list.append(df_temp['hperyn'][0])
    else:
        hperyn_list.append(0)
    
    if 'hpopyn' in list(df_temp.columns): # df_temp에 hpopyn 정보 없으면 0 append
        hpopyn_list.append(df_temp['hpopyn'][0])
    else:
        hpopyn_list.append(0)
    
hperyn_list, hpopyn_list

solution_df['hperyn'] = hperyn_list
solution_df['hpopyn'] = hpopyn_list

solution_df = solution_df[solution_df['hperyn'] > 0]
solution_df = solution_df[solution_df['hpopyn'] > 0]

solution_df.to_csv('gyeonggi_hospital_list.csv', index = False)

https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire?serviceKey=dmxRDNK9srMd2YgqgrNr94Qwm66GQ2qOeaR52ewsjheCoPNcAzknauVpluJX/EIgKY0Vo85YDDvCD23VN6MdvQ==&Q0=%EA%B2%BD%EA%B8%B0%EB%8F%84&pageNo=1&numOfRows=100


C:\Users\user\AppData\Local\Temp\ipykernel_21740\1748265710.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solution_df['hperyn'] = hperyn_list
C:\Users\user\AppData\Local\Temp\ipykernel_21740\1748265710.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solution_df['hpopyn'] = hpopyn_list


In [285]:
import pandas as pd
from urllib.parse import quote
from haversine import haversine
import ssl
from urllib.request import urlopen
import numpy as np

context=ssl.create_default_context()
context.set_ciphers("DEFAULT")

#########################################
# 1) 함수 선언하기                       #
#########################################
# 함수명 : find_hospital
# 매개변수 : special_m (중증질환명), lati, long (환자 위치)

def find_hospital(special_m, lati, long):
    #  [국립중앙의료원 - 전국응급의료기관 조회 서비스] 활용을 위한 개인 일반 인증키(Encoding) 저장

    key = "dmxRDNK9srMd2YgqgrNr94Qwm66GQ2qOeaR52ewsjheCoPNcAzknauVpluJX/EIgKY0Vo85YDDvCD23VN6MdvQ=="

    # 요청변수(Request Parameter)
    city = quote('경기도') # 주소(시도) : STAGE1
    city_detail = quote('성남시') # 주소(시군구) : STAGE2

    ############################################################
    # 2) 병원 리스트 csv 파일 불러오기 (daegu_hospital_list.csv) #
    ############################################################
    solution_df = pd.read_csv('gyeonggi_hospital_list.csv')

    ############################################################
    # 3) 병원 실시간 정보 가져오기                               #
    ############################################################

    # 응급실 실시간 가용병상 
    url = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire' + '?serviceKey=' + key + '&STAGE1=' + city + '&STAGE2=' + city_detail + '&pageNo=1&numOfRows=100'

    result = urlopen(url, context = context)
    emrRealtime_big = pd.read_xml(result, xpath='.//item')

    ## 응급실 실시간 가용병상 정보에서 기관코드(hpid), 응급실 병상수('hvec'), 수술실 수('hvoc') 정보만 추출하여 emRealtime_small 변수에 저장
    ## emrRealtime_big 중 [hpid, hvec, hvoc] 컬럼 활용
    emrRealtime_small = emrRealtime_big[['hpid', 'hvec', 'hvoc']]
    
    # solution_df와 emrRealtime_small 데이터프레임을 결합하여 solution_df에 저장
    solution_df = pd.merge(solution_df, emrRealtime_small)
    
    # 실시간 중증질환자 수용 가능 병원 조회
    url_acpt = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getSrsillDissAceptncPosblInfoInqire?serviceKey=' + key + '&STAGE1=' + city + '&pageNo=1&numOfRows=100'

    result = urlopen(url_acpt, context=context)
    emrAcpt_big = pd.read_xml(result, xpath='.//item')

    ## 다른 API함수와 다르게 기관코드 컬럼명이 다름 (hpid --> dutyName)
    ## 기관코드 컬렴명을 'hpid'로 일치화시키기 위해, 컬럼명을 변경함
    emrAcpt_big = emrAcpt_big.rename(columns = {'dutyName' : 'hpid'})
    
    ## 실시간 중증질환자 수용 가능 병원정보에서 필요한 정보만 추출하여 emrAcpt_small 변수에 저장
    ## emrAcpt 중 [hpid, MKioskTy1, MKioskTy2, MKioskTy3, MKioskTy4, MKioskTy5, MKioskTy7,MKioskTy8, MKioskTy10, MKioskTy11] 컬럼 확인
    emrAcpt_small = emrAcpt_big[['hpid', 'MKioskTy1', 'MKioskTy2', 'MKioskTy3', 'MKioskTy4', 'MKioskTy5', 'MKioskTy7','MKioskTy8', 'MKioskTy10', 'MKioskTy11']]


    # solution_df와 emrRealtime_small 데이터프레임을 결합하여 solution_df에 저장
    solution_df = pd.merge(solution_df, emrAcpt_small)
    
    ############################################################
    # 4) 자료 정비하기                                          #
    ############################################################

    # solution_df의 컬럼명 변경하기

    column_change = {  'hpid' : '병원코드',
                    'dutyName' : '병원명',
                    'dutyAddr' : '주소',
                    'dutyTel3' : '응급연락처',
                    'wgs84Lat' : '위도', 
                    'wgs84Lon' : '경도',
                    'hperyn' : '응급실수',
                    'hpopyn' : '수술실수',
                    'hvec' : '가용응급실수',
                    'hvoc' : '가용수술실수',
                    'MKioskTy1' : '뇌출혈', 
                    'MKioskTy2' : '뇌경색',
                    'MKioskTy3' : '심근경색',
                    'MKioskTy4' : '복부손상',
                    'MKioskTy5' : '사지접합',
                    'MKioskTy7' : '응급투석',
                    'MKioskTy8' : '조산산모',
                    'MKioskTy10' : '신생아',
                    'MKioskTy11' : '중증화상',                   
                    }

    solution_df = solution_df.rename(columns = column_change)
    
    # 중증질환 수용 가능 여부 데이터 중 정보 미제공, 불가능은 N로 변경 : replace
    solution_df = solution_df.replace(['정보미제공','불가능'], 'N')

    ## 응급실수/가용응급실수, 수술실수/가용수술실 수가 0보다 작은 경우는 비정상 데이터로 추정
    ## 0보다 작은 수는 0으로 변경
    solution_df.loc[solution_df['응급실수'] < 0, '응급실수'] = 0
    solution_df.loc[solution_df['가용응급실수'] < 0, '가용응급실수'] = 0
    solution_df.loc[solution_df['수술실수'] < 0, '수술실수'] = 0
    solution_df.loc[solution_df['가용수술실수'] < 0, '가용수술실수'] = 0

    # 응급실 가용율을 구하여 새로운 컬럼으로 추가하기
    # 컬렴명 : '응급실가용율'
    # 산식 : 가용 응급실수 / 응급실 수
    # 소수 둘째 자리까지 구하기 round() 활용
    solution_df['응급실가용율'] = round(solution_df['가용응급실수'] / solution_df['응급실수'],2)
    
    # 응급실 가용율이 1이 넘는 경우는 1로 대체
    solution_df.loc[solution_df['응급실가용율'] > 1, '응급실가용율'] = 1
    

    # 응급실 가용율에 따라 포화도 분류
    # 응급실 가용율 구분 단계 : ~0.1, 0.1 ~ 0.3, 0.3 ~ 0.6, 0.6 ~ 
    # 포화도 명칭 : ['불가', '혼잡', '보통', '원활']
    # pd.cut() 활용
    bin = [-np.inf, 0.1, 0.3, 0.6, np.inf]
    label = ['불가', '혼잡', '보통', '원활']

    solution_df['응급실포화도'] =  pd.cut(solution_df['응급실가용율'], bins = bin, labels = label)

    ############################################################
    # 5) 환자 수용 가능한 병원 구하기                            #  
    ############################################################

    # 매개변수 special_m로 받은 중증질환이 중증질환 리스트에 포함될 경우
    # 중증질환 리스트 :  ['뇌출혈', '뇌경색', '심근경색', '복부손상', '사지접합', '응급투석', '조산산모', '신생아','중증화상' ]

    if special_m in ['뇌출혈', '뇌경색', '심근경색', '복부손상', '사지접합', '응급투석', '조산산모', '신생아','중증화상']:
        distance_df = solution_df.loc[(solution_df[special_m] == 'Y') & (solution_df['응급실포화도'] != '불가'), :]
        

    # 매개변수 special_m 값이 중증질환 리스트에 포함이 안되는 경우
    else :
        distance_df = solution_df.loc[solution_df['응급실포화도'] != '불가', :]
        
        

    ############################################################
    # 6) 환자와 병원간 거리 구하기                               # 
    ############################################################
    # 환자와 병원간 거리를 저장할 빈 리스트 만들기 
    # 변수명 : distance
    distance = []
    

    #매개변수 lati, long으로 환자의 위치 정보 받기
    # 변수명 : patient
    patient = (lati, long)

    for idx, row in distance_df.iterrows():
        distance.append(round(haversine(patient, (row['위도'], row['경도']), 'km'),2))
        

    #distance_df의 '거리' 컬럼에, distance 값 저장하기
    distance_df['거리'] = distance

    ############################################################
    # 7) 거리 구간 구하기                                       #
    ############################################################
    # distancd_df의 '거리'에 따라 거리구분 분류 : cut()
    # '거리구분' 명칭 : ['2km이내', '5km이내', '10km이내', '10km이상']
    bin = [-np.inf, 2, 5, 10, np.inf]
    label = ['2km이내', '5km이내', '10km이내', '10km이상']

    distance_df['거리구분'] =  pd.cut(distance_df['거리'], bins = bin, labels = label)
    

    
    ############################################################
    # 8) 결과값 반환하기                                        #
    ############################################################
    return distance_df
    


In [293]:
# KAKAO API : KT 본사 Geocode

import requests

address = "경기도 성남시 분당구 불정로 90"

url = f"https://dapi.kakao.com/v2/local/search/address.json?query={address}"

headers = {"Authorization": "KakaoAK c19f52393c6d5bc2bb6547c1421fe954"}

response = requests.get(url, headers=headers)

data = response.json()

# 좌표 추출
if data.get("meta", {}).get("total_count", 0) > 0:
    first_result = data["documents"][0]
    my_lat = first_result["y"]
    my_lon = first_result["x"]
    print(f"위도: {my_lat}, 경도: {my_lon}")
else:
    print('Re-start.')


my_lat = round(float(my_lat), 2)
my_lon = round(float(my_lon), 2)

위도: 37.3587823498654, 경도: 127.114936252016


In [296]:
print(find_hospital('중증화상', my_lat, my_lon).sort_values(['거리구분', '응급실포화도', '거리'], ascending=[True, False, True]).to_markdown())

|    | 병원코드   | 병원명                 | 주소                                                | 응급연락처   |    위도 |    경도 |   응급실수 |   수술실수 |   가용응급실수 |   가용수술실수 | 뇌출혈   | 뇌경색   | 심근경색   | 복부손상   | 사지접합   | 응급투석   | 조산산모   | 신생아   | 중증화상   |   응급실가용율 | 응급실포화도   |   거리 | 거리구분   |
|---:|:-----------|:-----------------------|:----------------------------------------------------|:-------------|--------:|--------:|-----------:|-----------:|---------------:|---------------:|:---------|:---------|:-----------|:-----------|:-----------|:-----------|:-----------|:---------|:-----------|---------------:|:---------------|-------:|:-----------|
|  4 | A2100016   | 순천의료재단성남정병원 | 경기도 성남시 수정구 수정로 76, 지하3~10층 (수진동) | 031-750-6003 | 37.4396 | 127.129 |         12 |          4 |              8 |              5 | N        | N        | N          | Y          | N          | Y          | N          | N        | Y          |           0.67 | 원활           |   9.01 | 10km이내   |
|  3 | A2100011   | 성남중앙병원           | 

C:\Users\user\AppData\Local\Temp\ipykernel_21740\2314770454.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distance_df['거리'] = distance
C:\Users\user\AppData\Local\Temp\ipykernel_21740\2314770454.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distance_df['거리구분'] =  pd.cut(distance_df['거리'], bins = bin, labels = label)


## DX9반 화이팅!! DX여러분들 모두 수고하셨습니다!!